---
title: "SURVIVAL RATE ANALYSIS - UNSECURED PERSONAL LOAN PORTFOLIO"
---

## SURVIVAL RATE MODELING
### Analysis of Unsecured Personal Loan Portfolio Segemented by Credit Score, Rate and Original Loan Amount

This notebook demonstrates advanced survival analysis techniques using Kaplan-Meier and Cox regression models applied to unsecuredpersonal loan portfolio data. We progress from exploratory data analysis through univariate survival modeling to risk-adjusted survival estimates and competing risk analysis. Lastly, we'll compile findings for credit risk management and portfolio optimization applications.
Key Analyses:

1) Portfolio EDA and survival model readiness assessment

2) Kaplan-Meier survival estimation with censoring adjustments for young loans

3) Credit risk management applications (PD modeling, portfolio segmentation, loss forecasting)

------------------------------------------------------------------------------------------------------

In [12]:
# Import Packages for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [14]:
## Front loading functions for later use

In [15]:
def stratify_loan_dates(loan_data, start_date, end_date, n_start, n_loans, loan_per):
    # dtermine number loans in dataset to be stratified
    n_loans = int(n_loans * loan_per)
    n_loans_start = int(n_start + n_loans)
    
    loan_data['open_date'][n_start:int(n_loans_start)] = pd.date_range(start_date, end_date, periods=int(n_loans)) + pd.to_timedelta(np.random.randint(0, 30, int(n_loans)), unit='D')
    # Remove milliseconds, keep to seconds
    # Convert to datetime first, then remove time components
    loan_data['open_date'] = pd.to_datetime(loan_data['open_date']).dt.floor('D')
    #Backfill other date columns
    loan_data['open_month'] = loan_data['open_date'].dt.to_period('M')
    loan_data['open_year'] = loan_data['open_date'].dt.to_period('Y')
    loan_data['open_month_str'] = loan_data['open_month'].astype(str)

    return loan_data

In [16]:
def create_base_rates(loan_data, seed):
    """
    Create base rates for loans based on credit scores and time periods.
    Rates are inversely correlated with credit scores.
    """
    np.random.seed(seed)  # For reproducibility
    loan_data['open_date'] = pd.to_datetime(loan_data['open_date'])
    # Create rates that are inversely correlated with credit scores
    base_rates = []
    for i, (date, score) in enumerate(zip(loan_data['open_date'], loan_data['credit_score'])):
            
        # Base rate calculation using inverse relationship with credit score
        # Higher credit score = lower rate
        # Rate formula: Base rate influenced by credit score and time period
            
        # Credit score adjustment (inverse relationship)
        # Map credit scores (350-850) to rate adjustments
        # 850 score -> lowest rates, 350 score -> highest rates
        score_normalized = (score - 350) / (850 - 350)  # 0 to 1 scale
        credit_adjustment = -12 * score_normalized + 10  # High score = negative adjustment
            
        # Time-based base rate (market environment)
        if date < pd.to_datetime('2022-01-01'):
            time_base = 8.5  # Lower rates during COVID era
        elif date < pd.to_datetime('2022-04-01'):
            time_base = 9.0  # Slightly higher pre-increase
        else:
            time_base = 9.2  # Base before Fed increases
            
        # Combine factors
        calculated_rate = time_base + credit_adjustment
            
        # Add some realistic noise but maintain correlation
        noise = np.random.normal(0, 1.0)
        final_rate = calculated_rate + noise
            
        # Apply realistic bounds
        final_rate = max(3.99, min(29.99, final_rate))
        base_rates.append(round(final_rate, 2))
    return base_rates

In [17]:
def create_fed_funds_inspired_rates(df, rate_col='rate', date_col='open_date'):
    """
    Create rate increases that mirror Fed Funds rate increases from April 2022
    This creates a more realistic rate environment progression
    """
    
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Define key Fed rate increase dates and amounts (approximate)
    rate_increase_schedule = [
        ('2022-04-01', 0.25),   # First increase
        ('2022-05-01', 0.50),   # 0.5% increase
        ('2022-06-01', 0.75),   # 0.75% increase  
        ('2022-07-01', 0.75),   # Another 0.75%
        ('2022-09-01', 0.75),   # Continued increases
        ('2022-11-01', 0.75),   # Peak aggressive tightening
        ('2022-12-01', 0.50),   # Slowing pace
        ('2023-02-01', 0.25),   # Slower increases
        ('2023-05-01', 0.25),   # Final increases
        ('2023-07-01', 0.25),   # Last increase
    ]
    
    # Convert to DataFrame for easier manipulation
    rate_schedule = pd.DataFrame(rate_increase_schedule, 
                                columns=['date', 'increase'])
    rate_schedule['date'] = pd.to_datetime(rate_schedule['date'])
    rate_schedule['cumulative_increase'] = rate_schedule['increase'].cumsum()
    
    # Function to get cumulative rate increase for any date
    def get_rate_increase(loan_date):
        if loan_date < rate_schedule['date'].min():
            return 0.0
        
        applicable_increases = rate_schedule[rate_schedule['date'] <= loan_date]
        if len(applicable_increases) == 0:
            return 0.0
        
        return applicable_increases['cumulative_increase'].iloc[-1]
    
    # Apply rate increases
    df['fed_rate_increase'] = df[date_col].apply(get_rate_increase)
    df[f'{rate_col}_fed_adjusted'] = df[rate_col] + df['fed_rate_increase']
    df['rate'] = df[f'{rate_col}_fed_adjusted'].round(2)
    df.drop(columns=['fed_rate_increase', f'{rate_col}_fed_adjusted'], inplace=True, errors='ignore')
    
    return df

In [18]:
def bucket_loan_data(loan_data, col, bucket_col, bins, labels):
    """
    Bucket loan data into categories based on rate.
    """
    # Ensure 'rate' column exists
    if 'rate' not in loan_data.columns:
        raise ValueError("Data must contain 'rate' column for bucketing.")
    
    # Create buckets based on rate
    loan_data[bucket_col] = pd.cut(
                loan_data[col], 
                bins=bins, 
                labels=labels
    )
    return loan_data

In [42]:
def simulated_loan_data(loan_data, n_loans, start_date, mid_date, end_date, pre_fed_rate_increase, post_fed_rate_increase, seed):
    '''
    Generate realistic sample data for loans, including loan ID, credit score, interest rate, original amount, status, and dates.
    '''

    # Generate loan IDs
    loan_data['loan_id'] = range(1, n_loans + 1)
    loan_data['loan_id'] = loan_data['loan_id']+1000

    # Generate open dates with a random distribution within the specified range - stratify by pre and post fed rate increase
    loan_data = stratify_loan_dates(loan_data, start_date, mid_date, 0, n_loans, pre_fed_rate_increase)
    loan_data = stratify_loan_dates(loan_data, mid_date, end_date, int(n_loans*pre_fed_rate_increase), n_loans, post_fed_rate_increase)

    # Generate Original amounts with a log-normal distribution
    loan_data['orig_amount'] = np.random.lognormal(np.log(15000), 0.6, n_loans).clip(1000, 50000).round(2)
    # Value rounded
    # Round to nearest thousand
    loan_data['orig_amount'] = round(loan_data['orig_amount'] / 1000) * 1000


    # Generate credit scores with a normal distribution
    loan_data['credit_score'] =  np.random.normal(720, 80, n_loans).clip(300, 850).astype(int)

    # Assign status based on a random choice with probabilities
    loan_data['status'] = np.random.choice(['Open', 'Closed', 'Defaulted'], n_loans, p=[0.59, 0.32, 0.09])

    # Generate interest rates with a normal distribution
    loan_data['rate'] = create_base_rates(loan_data, seed)
    loan_data = create_fed_funds_inspired_rates(loan_data, 'rate', 'open_date')

    # Randomly assign a amtuirty date between 3 and 8 years from the open date
    loan_data['maturity_date'] = loan_data['open_date'] + pd.DateOffset(years = np.random.choice(range(3,9,1)))

    # Randomly assign a default date for defaulted loans, anywhere from 1 month to 3 years after the open date
    loan_data['default_date'] = loan_data.apply(
        lambda row: row['open_date'] + pd.DateOffset(months=np.random.randint(1, 36))
        if row['status'] == 'Defaulted' else pd.NaT,
        axis=1
    )

    # Create buckets for rate
    rate_bucket_bins = [0, 5, 8, 12, 15, 22]
    rate_bucket_labels = ['Low', 'Low-Med', 'Medium', 'Med-High', 'High']
    loan_data = bucket_loan_data(loan_data, 'rate', 'rate_bucket',
                bins=rate_bucket_bins,
                labels=rate_bucket_labels
                )

    # Create buckets for credit score
    score_bucket_bins=[0, 579,  619,  659, 719, 850]
    score_bucket_labels=['Deep Subprime', 'Subprime', 'Near-Prime', 'Prime', 'Super-Prime']
    loan_data = bucket_loan_data(loan_data, 'credit_score', 'score_bucket',
                bins=score_bucket_bins,
                labels=score_bucket_labels
                )

    # Create buckets for orignal amount
    orig_amount_bucket_bins = [0, 5000, 10000, 20000, 30000, 50000]
    orig_amount_bucket_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
    loan_data = bucket_loan_data(loan_data, 'orig_amount', 'orig_amount_bucket',
                bins=orig_amount_bucket_bins,
                labels=orig_amount_bucket_labels
                )

    # stratify 6 month credit score
    loan_data['6_month_credit_score'] = loan_data['credit_score'] + np.random.randint(-120, 30, n_loans)
    loan_data['6_month_credit_score'] = loan_data['6_month_credit_score'].clip(300, 850).astype(int)

    return loan_data

In [37]:
# Generate a random seed for reproducibility of the data set
seed = 42
# Establish Loan Pool
n_loans = 5000

# Generate Sample data distributions
pre_fed_rate_increase = 0.6
post_fed_rate_increase = 0.4
# Generate realistic sample data utilizing start and end dates
start_date = pd.to_datetime('2019-01-01')
mid_date = pd.to_datetime('2022-04-01')
end_date = pd.to_datetime('2025-5-31')

In [20]:
# Initialize dataframe for loan data
loan_data = pd.DataFrame(columns=['loan_id', 'open_date', 'credit_score', '6_month_credit_score', 'rate', 'orig_amount', 'status', 'rate_bucket', 'score_bucket', 'orig_amount_bucket', 'open_year', 'open_month', 'open_month_str', 'maturity_date', 'default_date'])

In [40]:
loan_data = simulated_loan_data(loan_data, n_loans, start_date, mid_date, end_date, pre_fed_rate_increase, post_fed_rate_increase, seed)

In [41]:
loan_data

,loan_id,open_date,credit_score,6_month_credit_score,rate,orig_amount,status,rate_bucket,score_bucket,orig_amount_bucket,open_year,open_month,open_month_str,maturity_date,default_date
0,1001,2019-01-18,775,779,8.80,6000.0,Open,Medium,Super-Prime,Low,2019,2019-01,2019-01,2022-01-18,NaT
1,1002,2019-01-12,676,621,10.54,31000.0,Open,Medium,Prime,Very High,2019,2019-01,2019-01,2022-01-12,NaT
2,1003,2019-01-16,630,539,12.43,17000.0,Closed,Med-High,Near-Prime,Medium,2019,2019-01,2019-01,2022-01-16,NaT
3,1004,2019-01-14,646,545,12.92,11000.0,Open,Med-High,Near-Prime,Medium,2019,2019-01,2019-01,2022-01-14,NaT
4,1005,2019-01-21,693,654,10.03,8000.0,Defaulted,Medium,Prime,Low,2019,2019-01,2019-01,2022-01-21,2019-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5996,2025-06-05,634,655,17.34,46000.0,Open,High,Near-Prime,Very High,2025,2025-06,2025-06,2028-06-05,NaT
4996,5997,2025-06-17,594,488,19.06,15000.0,Open,High,Subprime,Medium,2025,2025-06,2025-06,2028-06-17,NaT
4997,5998,2025-06-19,648,543,20.16,16000.0,Closed,High,Near-Prime,Medium,2025,2025-06,2025-06,2028-06-19,NaT
4998,5999,2025-06-04,714,595,16.27,24000.0,Open,High,Prime,High,2025,2025-06,2025-06,2028-06-04,NaT
